In [ ]:
import cv2
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define the dataset and data loader
train_folder = 'C:/Users/lolol/OneDrive/Документы/Дипломная работа/train'
test_folder =  'C:/Users/lolol/OneDrive/Документы/Дипломная работа/test'
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])
train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the neural network and define the loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# print(torch.__version__)

# Train the neural network
for epoch in range(15):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{15}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Evaluate the neural network on the test set
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            scores = model(data)
            _, predicted = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')

torch.save(model.state_dict(), 'person_detection_model.pth')
print('Модель создана')

In [ ]:
import cv2
# Set up the camera feed
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera feed
    ret, frame = cap.read()
    
    # Convert the frame from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Apply the transforms to the frame
    input_img = transform(frame)
    
    # Run the model on the input image
    outputs = model([input_img])
    
    # Get the predicted bounding boxes and labels
    boxes = outputs[0]['boxes'].detach().numpy()
    labels = outputs[0]['labels'].detach().numpy()
    
    # Draw the bounding boxes on the frame
    for i in range(len(boxes)):
        if labels[i] == 1:
            box = boxes[i]
            x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('frame', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera feed and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch
import torchvision
import numpy as np

# Define transforms to resize the images and normalize the pixel values
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load the COCO dataset
coco_dataset = torchvision.datasets.CocoDetection(root='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/train2017', annFile='C:/Users/lolol/OneDrive/Документы/СOCO Dataset/annotations/instances_train2017.json', transform=transform)

# Define the CNN architecture
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(64 * 28 * 28, 128)
        self.fc2 = torch.nn.Linear(128, 2)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the model
model = CNN()

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(coco_dataset, 0):
        # Get the inputs and labels
        inputs, annotations = data
        labels = torch.tensor([annotation['category_id'] for annotation in annotations])

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

# Test the model
model.eval()
total_correct = 0
total_images = 0
with torch.no_grad():
    for data in coco_dataset:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = 100 * total_correct / total_images
print('Accuracy: %.2f %%' % accuracy)

# Save the model
torch.save(model.state_dict(), 'person_detection_model.pth')
